Trying out dask to see if this is a better way to speed things up.

In [1]:
import pyrosetta
from pyrosetta.distributed.cluster import PyRosettaCluster

In [2]:
from dask.distributed import Client

client = Client()
client

#this does not work when I use the extension, which is a bummer

Client Scheduler: tcp://127.0.0.1:44267 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 24 Memory: 67.36 GB


In [3]:
%%writefile dask_docks.py

from pyrosetta.rosetta import *
from pyrosetta import *

def dask_dock(**kwargs):
    from pyrosetta.rosetta.protocols.flexpep_docking import FlexPepDockingProtocol
    
    import pyrosetta.distributed.io as io # Local import
    import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts # Local import
    
    pose = io.pose_from_file(kwargs["filepath"])
    #sf = create_score_function('docking') # no idea what this sf is...
    # Creating FlexPepDock protocol using init options
    fpdock = FlexPepDockingProtocol()
    fpdock.apply(pose)
    
    return pose

Overwriting dask_docks.py


Let's try docking some things.

In [4]:
from dask_docks import dask_dock
client.upload_file("dask_docks.py") # This sends a local file up to all worker nodes.

In [5]:
def create_kwargs():
    yield {
        "extra_options": "-out:level 300 -multithreading:total_threads 1", # Used by pyrosetta.init() on disributed workers
        "set_logging_handler": "interactive", # Used by pyrosetta.init() on disributed workers
        "s": "native_test.pdb",
    }

In [6]:
output_path = './decoys/'

PyRosettaCluster(
    tasks=create_kwargs,
    client=client,
    scratch_dir=output_path,
    output_path=output_path,
    nstruct=4, # Run the first user-provided PyRosetta protocol four times in parallel
).distribute(protocols=[dask_dock])

InvalidGitRepositoryError: There are untracked files in your git repository! If these are important for the simulation, you will not be able to reproduce your work! Commit the changes if appropriate, or add untracked files to your .gitignore file before running PyRosettaCluster!